In [120]:
import boto3
from datetime import datetime, timedelta
import json
import os
import pandas as pd
import sys
from tqdm import tqdm

base_dir = os.path.abspath('../')   # 메인 디렉토리 설정
sys.path.append(base_dir)
from src import aws_utils as aws
print(base_dir)

c:\Users\deachul.seo\Desktop\repo\yar_airdna


In [2]:
# SETUP

config = aws.load_config('../config.json')
aws_access_key_id = config['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['AWS_SECRET_ACCESS_KEY']
bucket_name = config['BUCKET_NAME']
base_prefix = config['BASE_PREFIX']
data_dir = config['DATA_DIRECTORY']
p_ym = config['P_YM']

output_date = aws.convert_date_format(p_ym)
bucket_dir = base_prefix + output_date

masked_id, masked_key = aws.display_config(aws_access_key_id, aws_secret_access_key)   # 민감정보 가리기

print('----------------------------CONFIG----------------------------')
print(f"AWS_ACCESS_KEY_ID: {masked_id}")
print(f"AWS_SECRET_ACCESS_KEY: {masked_key}")
print(f"BUCKET_NAME: {bucket_name}")
print(f"BUCKET_DIR: {bucket_dir}")
print(f"DATA_DIRECTORY: {data_dir}")
print(f"P_YM: {p_ym}")
print('---------------------------------------------------------------')

start_time = datetime.now()
print("START TIME:", start_time.time())


----------------------------CONFIG----------------------------
AWS_ACCESS_KEY_ID: AKI**************NHT
AWS_SECRET_ACCESS_KEY: RBI**********************************r45
BUCKET_NAME: airdna-prod-data-integration
BUCKET_DIR: yanolja/outbound/June 2024
DATA_DIRECTORY: data/
P_YM: 2024-06-01
---------------------------------------------------------------
START TIME: 09:04:26.360725


In [4]:
full_path = os.path.join(data_dir, output_date)
full_path

'../data/June 2024'

In [3]:
# AWS S3 스토리지 접속
print("Accessing AWS Storage...")
aws_access_key_id = config['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['AWS_SECRET_ACCESS_KEY']

s3 = aws.init_s3_client(aws_access_key_id, aws_secret_access_key)
print('Accessed S3 storage: ', s3)


Accessing AWS Storage...
Accessed S3 storage:  <botocore.client.S3 object at 0x000001ACA8735C90>


In [4]:
print('Start data download...')

aws.airdna_data_download(s3=s3, base_prefix=base_prefix, bucket_name=bucket_name, ml=output_date)

end_time = datetime.now()
print("END TIME:", start_time.time())

Start data download...


2024-08-05 17:22:39,941 - INFO - Directory set up at '../data/June 2024'
2024-08-05 17:22:39,942 - INFO - DOWNLOADING FILE: yanolja/outbound/June 2024/...
2024-08-05 17:22:40,439 - ERROR - Error handling file yanolja/outbound/June 2024/: [WinError 5] 액세스가 거부되었습니다: '../data/June 2024\\'
2024-08-05 17:22:40,441 - INFO - DOWNLOADING FILE: yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv...


In [3]:
### ANALYTICS (NEED UPDATE)
data_dir = '..\\data'   # remove in main
full_path = os.path.join(data_dir, output_date)
full_path

'..\\data\\June 2024'

In [37]:
output_date

'June 2024'

In [40]:
def assign_file_names(directory_path):
    """AirDNA 파일 읽기"""
    monthly_file = None
    property_file = None

    # List all files in the directory
    try:
        for entry in os.scandir(directory_path):
            if entry.is_file():  # Check if the entry is a file
                full_path = os.path.join(directory_path, entry.name)  # Construct full file path
                if 'monthly' in entry.name:
                    if monthly_file is not None:
                        raise Exception("More than one 'monthly' file found.")
                    monthly_file = full_path  # Use the full path
                elif 'property' in entry.name:
                    if property_file is not None:
                        raise Exception("More than one 'property' file found.")
                    property_file = full_path  # Use the full path

    except FileNotFoundError:
        print(f"The directory {directory_path} does not exist.")
        return None, None
    except PermissionError:
        print(f"Permission denied: unable to access {directory_path}.")
        return None, None
    except Exception as e:
        print(e)
        return None, None

    if monthly_file is None or property_file is None:
        print("One or both files could not be found.")
        return None, None
    
    return monthly_file, property_file


def generate_periods(yq):
    """분석에 포함할 년월 리스트를 반환"""
    year = int(yq[:4])
    quarter = int(yq[-1])

    # 분기
    quarter_to_months = {
        1: [1, 2, 3],
        2: [4, 5, 6],
        3: [7, 8, 9],
        4: [10, 11, 12]
    }

    # 작년 동기부터 포함
    start_month = quarter_to_months[quarter][0]
    start_date = datetime(year - 1, start_month, 1)

    periods = []
    
    # 작년 동기부터 해당 분기까지 데이터 반환
    current_date = start_date
    end_month = quarter_to_months[quarter][-1]
    while True:
        #  "yyyy-mm-dd" 형태로 반환 (airdna 포맷과 일치)
        periods.append(current_date.strftime('%Y-%m-%d'))
        
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)
        
        if current_date.year == year and current_date.month > end_month:
            break
    
    return periods


In [32]:
# READ DATA
monthly_file, property_file = assign_file_names(full_path)

print(f'READING: {property_file}')
df_property = pd.read_csv(property_file, dtype={56: 'str'})
print(f'READING: {monthly_file}')
df_performance = pd.read_csv(
    monthly_file,
    dtype={
        18: 'str',
        26: 'str', 
        27: 'str',
        28: 'str'  
    }
)


READING: ..\data\June 2024\Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv
READING: ..\data\June 2024\Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv


In [36]:
df_performance['City'].unique()

array(['Seoul', 'Jeju Island', nan, 'Jeollanam-do', 'Gyeongju-si',
       'Busan', 'Incheon', 'Seongnam', 'Gangwon-do', 'Jeollabuk-do',
       'Goyang', 'Daegu', 'Daejeon', 'Chungcheongbuk-do', 'Gwangju',
       'Suwon', 'Yongin', 'Ulsan', 'Gyeonggi-do', 'Bucheon', 'Anyang',
       'Chungcheongnam-do', 'Paengseong-eup', 'Uijeongbu', 'Chiang Mai'],
      dtype=object)

In [35]:
pd.to_datetime(df_performance['Reporting Month'])

0         2024-01-01
1         2024-02-01
2         2024-03-01
3         2024-04-01
4         2024-05-01
             ...    
8482686   2020-10-01
8482687   2020-11-01
8482688   2020-12-01
8482689   2021-01-01
8482690   2021-02-01
Name: Reporting Month, Length: 8482691, dtype: datetime64[ns]

In [41]:
# THIS VARIABLE SHOULD BE USED AS INPUT PARAM

yq = '2024q2'

In [45]:
periods = generate_periods(yq)
periods

['2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01']

In [177]:
def abnb_performance(data=None, month=None):
    occ_result = []
    adr_result = []
    revpar_result = []
    
    # Subset the data for active properties with available days and matching the reporting month
    subset = data[(data['Active'] == True) & (data['Available Days'] > 0) & (data['Reporting Month'] == month)]
    subset = subset[['Property ID', 'Occupancy Rate', 'Revenue (USD)', 'ADR (USD)', 'Number of Reservations', 'Reservation Days',
                     'Available Days', 'Blocked Days', 'State', 'City', 'Active', 'Revenue (Native)', 'ADR (Native)']]
    subset = subset[subset['City'].notna()]

    subset['occ'] = subset['Occupancy Rate']
    subset['adr(usd)'] = subset['Revenue (USD)'] / subset['Reservation Days']
    subset['adr(krw)'] = subset['Revenue (Native)'] / subset['Reservation Days']
    subset['adr(krw)'] = subset['adr(krw)'].fillna(0)
    
    lower_bound = subset['adr(krw)'].quantile(0.02)
    upper_bound = subset['adr(krw)'].quantile(0.98)
    subset = subset[(subset['adr(krw)'] > lower_bound) & (subset['adr(krw)'] < upper_bound)]
    subset['revpar'] = subset['occ'] * subset['adr(krw)']
    
    region_dict = {'Gangwon-do': '강원', 'Jeju Island': '제주', 'Seoul': '서울', 'Daegu': '경북권',
                   'Busan': '부산', 'Gyeongju-si': '경남권', 'Suwon': '경기권', 'Anyang': '경기권',
                   'Chungcheongbuk-do': '충청권', 'Incheon': '경기권', 'Daejeon': '충청권', 'Gwangju': '전라권',
                   'Gyeonggi-do': '경기권', 'Seongnam': '경기권', 'Paengseong-eup': '경기권', 'Bucheon': '경기권',
                   'Jeollanam-do': '전라권', 'Jeollabuk-do': '전라권', 'Yongin': '경기권', 'Chungcheongnam-do': '충청권',
                   'Ulsan': '경남권', 'Goyang': '경기권', 'Uijeongbu': '경기권'}
    subset['Region'] = subset['City'].replace(region_dict)
    subset = subset.reset_index()

    for rg in subset['Region'].unique():
        occ_result.append([rg, subset[subset['Region'] == rg]['occ'].mean()])
        adr_result.append([rg, subset[subset['Region'] == rg]['adr(krw)'].mean()])
        revpar_result.append([rg, subset[subset['Region'] == rg]['revpar'].mean()])

    occ_result.append(['전국', subset['occ'].mean()])
    adr_result.append(['전국', subset['adr(krw)'].mean()])
    revpar_result.append(['전국', subset['revpar'].mean()])

    occ_df = pd.DataFrame(occ_result).transpose()
    adr_df = pd.DataFrame(adr_result).transpose()
    revpar_df = pd.DataFrame(revpar_result).transpose()

    # 인덱스를 연월로 변환
    year_month = f"{month[:4]}.{month[5:7]}"
    
    occ_df.columns = occ_df.iloc[0]
    adr_df.columns = adr_df.iloc[0]
    revpar_df.columns = revpar_df.iloc[0]
    
    occ_df = occ_df[1:].set_index([[year_month]])
    adr_df = adr_df[1:].set_index([[year_month]])
    revpar_df = revpar_df[1:].set_index([[year_month]])
    
    col_order = ['전국', '강원', '경기권', '경남권', '경북권', '부산', '서울', '전라권', '제주', '충청권']
    occ_df = occ_df.reindex(columns=col_order)
    adr_df = adr_df.reindex(columns=col_order)
    revpar_df = revpar_df.reindex(columns=col_order)

    return occ_df, adr_df, revpar_df


def quarter_from_month(date_str):
    """연월정보에서 분기정보 추출"""
    year, month = map(int, date_str.split('.'))
    quarter = (month - 1) // 3 + 1
    return f"{year}.Q{quarter}"


def aggregate_by_quarter(df):
    """월별데이터 분기별로 변환"""
    df2 = df.copy()
    df2.index = [quarter_from_month(idx) for idx in df.index]
    quarterly_df = df2.groupby(df2.index).mean()
    return quarterly_df


def reset_index_and_rename(df):
    """'시점' 칼럼 생성"""
    df_reset = df.reset_index().rename(columns={'index': '시점'})
    return df_reset


In [151]:
# occ_dfs = []
# adr_dfs = []
# revpar_dfs = []

# for period in tqdm(periods, desc='RUNNING MONTHLY ANALYSIS...'):
#     occ_df, adr_df, revpar_df = abnb_performance(data=df_performance, month=period)

#     occ_dfs.append(occ_df)
#     adr_dfs.append(adr_df)
#     revpar_dfs.append(revpar_df)

# print('AGGREGATING QUARTERLY DATA...')
# full_occ_df = pd.concat(occ_dfs)
# full_adr_df = pd.concat(adr_dfs)
# full_revpar_df = pd.concat(revpar_dfs)

# # 분기 데이터 정리
# quarterly_occ_df = aggregate_by_quarter(full_occ_df)
# quarterly_adr_df = aggregate_by_quarter(full_adr_df)
# quarterly_revpar_df = aggregate_by_quarter(full_revpar_df)

# # '시점' 칼럼 생성
# full_occ_df = reset_index_and_rename(full_occ_df)
# full_adr_df = reset_index_and_rename(full_adr_df)
# full_revpar_df = reset_index_and_rename(full_revpar_df)

# quarterly_occ_df = reset_index_and_rename(quarterly_occ_df)
# quarterly_adr_df = reset_index_and_rename(quarterly_adr_df)
# quarterly_revpar_df = reset_index_and_rename(quarterly_revpar_df)

# # 분기별/월별 테이블 결합 (첫 분기 월별 데이터 제거 후) --> 분기동향 포맷으로 설정
# final_adr_df = pd.concat([quarterly_adr_df, full_adr_df.iloc[3:]])
# final_occ_df = pd.concat([quarterly_occ_df, full_occ_df.iloc[3:]])
# final_revpar_df = pd.concat([quarterly_revpar_df, full_revpar_df.iloc[3:]])

# # 단위 반올림
# final_adr_df.iloc[:, 1:] = final_adr_df.iloc[:, 1:].astype(float).round(0).astype(int)
# final_revpar_df.iloc[:, 1:] = final_revpar_df.iloc[:, 1:].astype(float).round(0).astype(int)
# final_occ_df.iloc[:, 1:] = (final_occ_df.iloc[:, 1:].astype(float) * 100).round(1)

# # 보고서 테이블 형식으로 변환

# final_adr_df['Index'] = 'ADR'
# final_occ_df['Index'] = 'OCC'
# final_revpar_df['Index'] = 'RevPAR'

# combined_df = pd.concat([final_adr_df, final_occ_df, final_revpar_df], ignore_index=True)
# combined_df.set_index('Index', inplace=True)
# combined_df = combined_df.reset_index()


RUNNING MONTHLY ANALYSIS...: 100%|██████████| 15/15 [00:07<00:00,  2.13it/s]

AGGREGATING QUARTERLY DATA...


In [186]:
# ## refactoring (v2) -- use this

# def run_monthly_analysis(data, periods):
#     """월별 데이터 정리"""
#     occ_dfs = []
#     adr_dfs = []
#     revpar_dfs = []

#     for period in tqdm(periods, desc='RUNNING MONTHLY ANALYSIS...'):
#         occ_df, adr_df, revpar_df = abnb_performance(data, month=period)
#         occ_dfs.append(occ_df)
#         adr_dfs.append(adr_df)
#         revpar_dfs.append(revpar_df)

#     return occ_dfs, adr_dfs, revpar_dfs


# def aggregate_quarterly_data(occ_dfs, adr_dfs, revpar_dfs):
#     """분기별 데이터 정리"""
#     full_occ_df = pd.concat(occ_dfs)
#     full_adr_df = pd.concat(adr_dfs)
#     full_revpar_df = pd.concat(revpar_dfs)

#     quarterly_occ_df = aggregate_by_quarter(full_occ_df)
#     quarterly_adr_df = aggregate_by_quarter(full_adr_df)
#     quarterly_revpar_df = aggregate_by_quarter(full_revpar_df)

#     return quarterly_occ_df, quarterly_adr_df, quarterly_revpar_df, full_occ_df, full_adr_df, full_revpar_df


# def reset_index_and_rename(df):
#     """Resets the index of the DataFrame and renames the index column to '시점'."""
#     df_reset = df.reset_index().rename(columns={'index': '시점'})
#     return df_reset


# def concatenate_and_refine_data(quarterly_df, full_df, is_occ=False):
#     """OCC는 소수점 1째 자리까지 반올림, 다른 테이블은 정수로 반올림"""
#     refined_df = pd.concat([quarterly_df, full_df.iloc[3:]])

#     if not is_occ: 
#         # adr, revpar
#         refined_df.iloc[:, 1:] = refined_df.iloc[:, 1:].astype(float).round(0).astype(int)
#     elif is_occ:
#         # occ
#         refined_df.iloc[:, 1:] = refined_df.iloc[:, 1:].astype(float) * 100
#         refined_df.iloc[:, 1:] = refined_df.iloc[:, 1:].astype(float).round(1)    # NEED TO FIX: ROUNDING NOT WORKING
    
#     return refined_df


# def finalize_report_tables(final_adr_df, final_occ_df, final_revpar_df):
#     final_adr_df['Index'] = 'ADR'
#     final_occ_df['Index'] = 'OCC'
#     final_revpar_df['Index'] = 'RevPAR'
    
#     combined_df = pd.concat([final_adr_df, final_occ_df, final_revpar_df], ignore_index=True)
#     combined_df.set_index('Index', inplace=True)
#     combined_df = combined_df.reset_index()

#     # 테이블값 string으로 변환 

#     combined_df = combined_df.astype('str')

#     return combined_df


# # Main script to execute functions
# def analyze_abnb(data, periods):
#     occ_dfs, adr_dfs, revpar_dfs = run_monthly_analysis(data, periods)
    
#     quarterly_occ_df, quarterly_adr_df, quarterly_revpar_df, full_occ_df, full_adr_df, full_revpar_df = aggregate_quarterly_data(occ_dfs, adr_dfs, revpar_dfs)
    
#     # Apply reset_index_and_rename if needed here
#     full_occ_df = reset_index_and_rename(full_occ_df)
#     full_adr_df = reset_index_and_rename(full_adr_df)
#     full_revpar_df = reset_index_and_rename(full_revpar_df)
    
#     quarterly_occ_df = reset_index_and_rename(quarterly_occ_df)
#     quarterly_adr_df = reset_index_and_rename(quarterly_adr_df)
#     quarterly_revpar_df = reset_index_and_rename(quarterly_revpar_df)
    
#     final_adr_df = concatenate_and_refine_data(quarterly_adr_df, full_adr_df)
#     final_occ_df = concatenate_and_refine_data(quarterly_occ_df, full_occ_df, is_occ=True)

#     # return final_occ_df

#     final_revpar_df = concatenate_and_refine_data(quarterly_revpar_df, full_revpar_df)
#     report_table = finalize_report_tables(final_adr_df, final_occ_df, final_revpar_df)

#     print(f'PERIODS:')
#     for p in periods:
#         print(f'- {p}')
    
#     print('COMPLETE.')

#     return report_table


In [187]:

# Run the main function with the appropriate parameters

final_table = analyze_abnb(df_performance, periods)
final_table



RUNNING MONTHLY ANALYSIS...: 100%|██████████| 15/15 [00:06<00:00,  2.14it/s]

PERIODS:
- 2023-04-01
- 2023-05-01
- 2023-06-01
- 2023-07-01
- 2023-08-01
- 2023-09-01
- 2023-10-01
- 2023-11-01
- 2023-12-01
- 2024-01-01
- 2024-02-01
- 2024-03-01
- 2024-04-01
- 2024-05-01
- 2024-06-01
COMPLETE.


,Index,시점,전국,강원,경기권,경남권,경북권,부산,서울,전라권,제주,충청권
0,ADR,2023.Q2,128975,144929,115794,161991,96730,124122,110726,136464,129132,123502
1,ADR,2023.Q3,144707,179423,127117,181059,98774,142637,113383,153403,144112,137057
2,ADR,2023.Q4,137931,158733,124970,171894,104745,135157,122232,144144,135928,133855
3,ADR,2024.Q1,132277,152911,120984,169657,101212,128520,115044,138738,134470,126679
4,ADR,2024.Q2,135532,153000,122769,169405,99290,125096,120432,142275,140508,136046
5,ADR,2023.07,145135,182831,126384,176999,93482,142253,111739,155390,143778,134976
6,ADR,2023.08,150236,193839,129540,190061,97688,154081,111443,159550,148945,140989
7,ADR,2023.09,138751,161598,125426,176116,105153,131578,116967,145270,139612,135205
8,ADR,2023.10,137878,158264,123953,172544,105476,135686,119215,143052,136872,134495
9,ADR,2023.11,130833,149402,118093,163762,96270,127641,114861,139338,131145,131318


In [2]:
## TEST CODE

import boto3
from datetime import datetime, timedelta
import json
import os
import pandas as pd
import sys
from tqdm import tqdm

base_dir = os.path.abspath('../')   # 메인 디렉토리 설정
sys.path.append(base_dir)
from src import aws_utils as aws
from src import analysis as ans
print(base_dir)

# SETUP

config = aws.load_config('../config.json')
aws_access_key_id = config['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['AWS_SECRET_ACCESS_KEY']
bucket_name = config['BUCKET_NAME']
base_prefix = config['BASE_PREFIX']
data_dir = config['DATA_DIRECTORY']
p_ym = config['P_YM']
yq = config['YQ']

output_date = aws.convert_date_format(p_ym)
bucket_dir = base_prefix + output_date
full_path = os.path.join(data_dir, output_date)


masked_id, masked_key = aws.display_config(aws_access_key_id, aws_secret_access_key)   # 민감정보 가리기

print('----------------------------CONFIG----------------------------')
print(f"AWS_ACCESS_KEY_ID: {masked_id}")
print(f"AWS_SECRET_ACCESS_KEY: {masked_key}")
print(f"BUCKET_NAME: {bucket_name}")
print(f"BUCKET_DIR: {bucket_dir}")
print(f"DATA_DIRECTORY: {full_path}")
print(f"P_YM: {p_ym}")
print(f"QUARTER: {yq}")
print('---------------------------------------------------------------')

start_time = datetime.now()
print("START TIME:", start_time.time())


c:\Users\deachul.seo\Desktop\repo\yar_airdna
----------------------------CONFIG----------------------------
AWS_ACCESS_KEY_ID: AKI**************NHT
AWS_SECRET_ACCESS_KEY: RBI**********************************r45
BUCKET_NAME: airdna-prod-data-integration
BUCKET_DIR: yanolja/outbound/June 2024
DATA_DIRECTORY: data/June 2024
P_YM: 2024-06-01
QUARTER: 2024q2
---------------------------------------------------------------
START TIME: 21:28:27.333908


In [3]:
data_dir = '..\\data'   # hard coding for testing


In [4]:
full_path = os.path.join(data_dir, output_date)
full_path

'..\\data\\June 2024'

In [6]:
print('Reading abnb data...')

monthly_file, property_file = ans.assign_file_names(full_path)
# print(f'READING: {property_file}')
# df_property = pd.read_csv(property_file, dtype={56: 'str'})
print(f'READING: {monthly_file}')
df_performance = pd.read_csv(
    monthly_file,
    dtype={
        18: 'str',
        26: 'str', 
        27: 'str',
        28: 'str'  
    }
)
print('DATA IMPORT COMPLETE.')

periods = ans.generate_periods(yq)
print(f'PERIODS:')
for p in periods:
    print(f'- {p}')

full_table = ans.analyze_abnb(df_performance, periods)


filename = 'abnb_' + yq + '.csv'
filepath = os.path.join(full_path, filename)
ans.save_data(full_table, filepath)



Reading abnb data...
READING: ..\data\June 2024\Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv
DATA IMPORT COMPLETE.
PERIODS:
- 2023-04-01
- 2023-05-01
- 2023-06-01
- 2023-07-01
- 2023-08-01
- 2023-09-01
- 2023-10-01
- 2023-11-01
- 2023-12-01
- 2024-01-01
- 2024-02-01
- 2024-03-01
- 2024-04-01
- 2024-05-01
- 2024-06-01


RUNNING MONTHLY ANALYSIS...: 100%|██████████| 15/15 [00:11<00:00,  1.27it/s]

COMPLETE.
ENCODING: cp949
DATA SAVED: ..\data\June 2024\abnb_2024q2.csv


In [9]:
full_table

,Index,시점,전국,강원,경기권,경남권,경북권,부산,서울,전라권,제주,충청권
0,ADR,2023.Q2,128975,144929,115794,161991,96730,124122,110726,136464,129132,123502
1,ADR,2023.Q3,144707,179423,127117,181059,98774,142637,113383,153403,144112,137057
2,ADR,2023.Q4,137931,158733,124970,171894,104745,135157,122232,144144,135928,133855
3,ADR,2024.Q1,132277,152911,120984,169657,101212,128520,115044,138738,134470,126679
4,ADR,2024.Q2,135532,153000,122769,169405,99290,125096,120432,142275,140508,136046
5,ADR,2023.07,145135,182831,126384,176999,93482,142253,111739,155390,143778,134976
6,ADR,2023.08,150236,193839,129540,190061,97688,154081,111443,159550,148945,140989
7,ADR,2023.09,138751,161598,125426,176116,105153,131578,116967,145270,139612,135205
8,ADR,2023.10,137878,158264,123953,172544,105476,135686,119215,143052,136872,134495
9,ADR,2023.11,130833,149402,118093,163762,96270,127641,114861,139338,131145,131318


In [11]:
# def save_data(df, filepath, codec='cp949'):

#     df.to_csv(filepath, encoding=codec)
#     print(f'ENCODING: {codec}')
#     print(f'DATA SAVED: {filepath}')

In [14]:
filename = 'abnb_' + yq + '.csv'
filepath = os.path.join(full_path, filename)

ans.save_data(full_table, filepath)

'..\\data\\June 2024\\abnb_2024q2.csv'

In [13]:
full_path


'..\\data\\June 2024'

In [9]:
full_path

'..\\data\\June 2024'

In [ ]:

# main code update template

import argparse
from datetime import datetime
from src import aws_utils as aws

def main():
    parser = argparse.ArgumentParser(description="Run the Airdna main process.")
    parser.add_argument('--skip-download', action='store_true',
                        help='Skip the downloading of data from AWS S3')

    args = parser.parse_args()

    config = aws.load_config()
    output_date = aws.convert_date_format(config['P_YM'])
    bucket_dir = config['BASE_PREFIX'] + output_date

    if not args.skip_download:
        print("Accessing AWS Storage and downloading data...")
        s3 = aws.init_s3_client(config['AWS_ACCESS_KEY_ID'], config['AWS_SECRET_ACCESS_KEY'])
        aws.airdna_data_download(s3=s3, base_prefix=config['BASE_PREFIX'], bucket_name=config['BUCKET_NAME'], ml=output_date)
    else:
        print("Skipping data download...")

    # Proceed with the analytics part
    print("Starting analytics...")
    # Add your analytics module function call here
    analytics_results = analytics_module.run_analytics()

if __name__ == '__main__':
    main()
